In [1]:
from azureml.core import Workspace, Experiment, Environment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="azure_ML_engineer")
env = Environment.get(workspace=ws, name="AzureML-Tutorial")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group,
      'Environment: ', env,
       sep = '\n')

run = exp.start_logging()

Workspace name: mldevtest
Azure region: eastus2
Subscription id: ac401033-05b1-43d5-a5ea-e5dcb9c75b49
Resource group: rg-devtest-databricks-01
Environment: 
Environment(Name: AzureML-Tutorial,
Version: 67)


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "MLDevTestCluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                            max_nodes=2)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

Found existing cluster, use it.


In [18]:
from azureml.widgets import RunDetails
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import numpy as np

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        'C': uniform(0,1),
        'max_iter': choice(100, 250, 500)
    }
)

# # Specify a Policy
policy = BanditPolicy(slack_amount=0.10)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create an estimator by running train.py
src = ScriptRunConfig(
    source_directory=".", 
    script="train.py", 
    compute_target=cpu_cluster,
    environment=env
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src, 
    hyperparameter_sampling=ps, 
    primary_metric_name='Accuracy', 
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
    max_total_runs=4, 
    policy=policy
) 

In [19]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)
print("Run submitted for execution.")
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)


Run submitted for execution.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [21]:
from azureml.core import Model

# Get your best run and save the model from that run.
best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()

print(best_hyperdrive_run.get_metrics())

best_hyperdrive_run.register_model(model_name = 'hyperdrive_bankmarketing', model_path='outputs/hyperdrive_model.joblib', model_framework=Model.Framework.SCIKITLEARN)

# joblib.dump(best_hyperdrive_run, 'hyperdrive_bankmarketing.joblib') 
# best_hyperdrive_run = joblib.load('hyperdrive_bankmarketing.joblib') 


{'Regularization Strength:': 0.25929149684660246, 'Max iterations:': 250, 'Accuracy': 0.9154779969650987}


ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path outputs/hyperdrive_model.joblib in the set of files uploaded to the run: ['azureml-logs/55_azureml-execution-tvmps_da9f08b666440a99cd6337be0ad3f800b6f41cc568ef5231f68dbd53b02fb575_p.txt', 'azureml-logs/65_job_prep-tvmps_da9f08b666440a99cd6337be0ad3f800b6f41cc568ef5231f68dbd53b02fb575_p.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_da9f08b666440a99cd6337be0ad3f800b6f41cc568ef5231f68dbd53b02fb575_p.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/106_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log']
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Could not locate the provided model_pat

In [34]:
from azureml.data.dataset_factory import TabularDatasetFactory
from train import clean_data

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

# Use the clean_data function to clean your data.
# Do not split into train and test sets since AutoML takes care of that for you
x, y = clean_data(ds)

In [38]:
import pandas as pd

# Join x and y back together for the AutoML run
# training_data = pd.concat([x, y], axis=1)
training_data = x.join(y)
print(x.shape)
print(y.shape)
print(training_data.shape)

# Save the data so that it could loaded into a TabularDataset
training_data.to_csv('./training/training_data.csv')

from azureml.core import Dataset

# get the datastore to upload prepared data
datastore = ws.get_default_datastore()
# upload the local file from src_dir to the target_path in datastore
datastore.upload(src_dir='./training', target_path='./training')

(32950, 39)
(32950,)
(32950, 40)
Uploading an estimated of 2 files
Target already exists. Skipping upload for training/training_data
Uploading ./training/training_data.csv
Uploaded ./training/training_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_5aa0cf21a4b645759ea22b173665af16

In [41]:
from azureml.train.automl import AutoMLConfig

TabularDataset_training_data = TabularDatasetFactory.from_delimited_files(datastore.path('./training/training_data'))

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=TabularDataset_training_data,
    label_column_name='y',
    n_cross_validations=4,
    compute_target = cpu_cluster
)

In [42]:
# Submit your automl run
automl_run = exp.submit(automl_config)
RunDetails(automl_run).show()
print("Run submitted for execution.")

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
azure_ML_engineer,AutoML_a2a15dab-b1af-463e-b940-d399729dc5a0,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Run submitted for execution.


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [58]:
# Retrieve and save your best automl model.
best_automl_run, fitted_model = automl_run.get_output()

print(best_automl_run.get_metrics())

# best_hyperdrive_run.register_model(model_name = 'automl_bankmarketing', model_path = 'outputs/automl_bankmarketing.model')

joblib.dump(fitted_model, 'automl_bankmarketing.joblib') 
# fitted_model = joblib.load('automl_bankmarketing.joblib') 

Package:azureml-automl-runtime, training version:1.29.0, current version:1.27.0
Package:azureml-core, training version:1.29.0, current version:1.27.0
Package:azureml-dataprep, training version:2.15.1, current version:2.14.2
Package:azureml-dataprep-rslex, training version:1.13.0, current version:1.12.1
Package:azureml-dataset-runtime, training version:1.29.0, current version:1.27.0
Package:azureml-defaults, training version:1.29.0, current version:1.27.0
Package:azureml-interpret, training version:1.29.0, current version:1.27.0
Package:azureml-mlflow, training version:1.29.0, current version:1.27.0
Package:azureml-pipeline-core, training version:1.29.0, current version:1.27.0
Package:azureml-telemetry, training version:1.29.0, current version:1.27.0
Package:azureml-train-automl-client, training version:1.29.0, current version:1.27.0
Package:azureml-train-automl-runtime, training version:1.29.0, current version:1.27.0


{'AUC_weighted': 0.948456969506999, 'recall_score_weighted': 0.9179362670713201, 'average_precision_score_macro': 0.8274742506450433, 'recall_score_macro': 0.7772115321471402, 'precision_score_micro': 0.9179362670713201, 'average_precision_score_weighted': 0.9561407626355299, 'recall_score_micro': 0.9179362670713201, 'AUC_micro': 0.9810065925057738, 'f1_score_micro': 0.9179362670713201, 'f1_score_macro': 0.7867368474856672, 'balanced_accuracy': 0.7772115321471402, 'log_loss': 0.24458793723358588, 'AUC_macro': 0.948456969506999, 'f1_score_weighted': 0.9165335111278882, 'average_precision_score_micro': 0.9817762890035919, 'matthews_correlation': 0.5742726941399052, 'accuracy': 0.9179362670713201, 'precision_score_macro': 0.7974981655411582, 'norm_macro_recall': 0.5544230642942802, 'precision_score_weighted': 0.9154500835649009, 'weighted_accuracy': 0.9528905679374194, 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_a2a15dab-b1af-463e-b940-d399729dc5a0_22/confusion_matrix'

['automl_bankmarketing.joblib']